# Whisper 음성 전사 도구

**간단한 4단계 실행:**
1. 패키지 설치
2. Google Drive 마운트
3. 설정 및 함수 정의
4. 실행

결과는 Google Drive의 `whisper_notion_pipeline/output` 폴더에 저장됩니다.

In [ ]:
# 1단계: 패키지 설치 및 GPU 확인
!nvidia-smi
%pip install -q openai-whisper

import torch
print(f"\nGPU 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")

In [ ]:
# 2단계: Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

import os
WORK_DIR = '/content/drive/MyDrive/whisper_notion_pipeline'
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(f'{WORK_DIR}/input', exist_ok=True)
os.makedirs(f'{WORK_DIR}/output', exist_ok=True)

print(f"작업 디렉토리: {WORK_DIR}")
print("✓ input 폴더에 오디오 파일을 업로드하세요")

In [ ]:
# 3단계: 설정 및 함수 정의
import whisper
import json
import time
from pathlib import Path
import glob

# Whisper 모델 설정
MODEL_NAME = "base"  # tiny, base, small, medium, large
LANGUAGE = "ko"  # 언어 설정

def process_audio_file(audio_path):
    """오디오 파일을 텍스트로 변환"""
    audio_path = Path(audio_path)
    
    # 이미 처리된 파일 확인
    output_dir = Path(f"{WORK_DIR}/output/{audio_path.stem}")
    if output_dir.exists() and (output_dir / "transcript.txt").exists():
        print(f"⏭️  이미 처리됨: {audio_path.name}")
        return None
    
    # 출력 디렉토리 생성
    output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\n{'='*50}")
    print(f"🎤 처리 시작: {audio_path.name}")
    print(f"{'='*50}")
    
    # Whisper 모델 로드 및 전사
    print("🔄 Whisper 모델 로드 중...")
    model = whisper.load_model(MODEL_NAME, device="cuda" if torch.cuda.is_available() else "cpu")
    print(f"✓ 모델 로드 완료: {MODEL_NAME}")
    
    print(f"\n🎧 오디오 전사 시작...")
    start_time = time.time()
    
    # 진행 상황을 보여주기 위해 verbose=True 추가
    result = model.transcribe(
        str(audio_path), 
        language=LANGUAGE,
        verbose=True,  # 진행 상황 표시
        fp16=torch.cuda.is_available()  # GPU 사용 시 더 빠른 처리
    )
    
    elapsed_time = time.time() - start_time
    print(f"\n✓ 전사 완료! (소요 시간: {elapsed_time:.1f}초)")
    
    # 결과 저장
    print(f"\n💾 결과 저장 중...")
    with open(output_dir / "transcript.txt", "w", encoding="utf-8") as f:
        f.write(result['text'])
    
    with open(output_dir / "report.json", "w", encoding="utf-8") as f:
        json.dump({
            "file_name": audio_path.name,
            "text": result['text'],
            "language": result['language'],
            "duration": f"{elapsed_time:.1f}초"
        }, f, ensure_ascii=False, indent=2)
    
    print(f"✓ 저장 완료: {output_dir}")
    print(f"\n📝 전사 결과 미리보기:")
    print(f"   {result['text'][:100]}..." if len(result['text']) > 100 else f"   {result['text']}")
    
    return result

print("✓ 설정 완료")

In [ ]:
# 4단계: 실행 - 모든 오디오 파일 처리
audio_extensions = ['*.mp3', '*.m4a', '*.wav', '*.flac', '*.ogg']
audio_files = []

for ext in audio_extensions:
    audio_files.extend(glob.glob(f"{WORK_DIR}/input/{ext}"))

if audio_files:
    print(f"🔍 발견된 파일: {len(audio_files)}개")
    for i, file in enumerate(audio_files, 1):
        print(f"   {i}. {os.path.basename(file)}")
    
    print(f"\n🚀 전체 처리 시작...\n")
    total_start = time.time()
    
    for idx, audio_file in enumerate(audio_files, 1):
        print(f"\n[{idx}/{len(audio_files)}] 파일 처리")
        process_audio_file(audio_file)
    
    total_time = time.time() - total_start
    print(f"\n{'='*50}")
    print(f"✅ 모든 처리 완료!")
    print(f"   총 소요 시간: {total_time:.1f}초")
    print(f"   평균 처리 시간: {total_time/len(audio_files):.1f}초/파일")
    print(f"{'='*50}")
else:
    print("❌ input 폴더에 오디오 파일이 없습니다.")
    print(f"   경로: {WORK_DIR}/input/")